In [81]:
from bs4 import BeautifulSoup
import urllib.request
import re
import os
import shutil
import json

In [82]:
def prepare_text(name, doc):
    doc_name = doc.contents[0]
    doc_url = re.search(r'<a href="(.*?)">', str(doc)).group(1)

    url = 'http://magazines.russ.ru' + str(doc_url)
    response = urllib.request.urlopen(url)
    page = response.read().decode('utf-8')
    soup = BeautifulSoup(page, "html.parser")

    for script in soup(["script", "style"]):
        script.extract()    # rip it out
        text = soup.get_text()

    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)
    text = text[text.find('Опубликовано в журнале') + 28:text.find('Версия для печати')]
    
    return text

In [83]:
def parse_page(page, directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

    soup = BeautifulSoup(page, "html.parser")
    a_data = soup.find_all("div",{"class":"jrn_right col-xs-10"})

    a = a_data[0].find_all('a')
    n = a_data[0].find_all('strong')
    
    i = 0
    for name, doc in zip(n, a):
        text = prepare_text(name, doc)
        
        f = open(directory + 'article' + str(i).zfill(2) + '.txt', 'w+', encoding="utf-8")
        f.write(text)
        f.close()
        
        i += 1
        
    return text

In [86]:
def get_data(path='.', 
             url='http://magazines.russ.ru/ural/%d/%d',
             year_start = 2016, month_start = 1,
             year_end = 2017, month_end = 12):
    
    data = []
    
    month_max = 12

    for year in range(year_start, year_end + 1):
        for month in range(month_start, month_max + 1):
            if (year == year_end and month > month_end):
                break

            response = urllib.request.urlopen(url % (year, month))
            page = response.read().decode('utf-8')
            data.append(page)
            
            directory = path + '/plain/' + str(year) + '/' + str(month).zfill(2) + '/'
            parse_page(page, directory)
    return

In [87]:
a = get_data('zhurnalnii_stol_1', 'http://magazines.russ.ru/ural/%d/%d', 2016, 1, 2018, 6)